In [19]:
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import matplotlib.pyplot as plt
import plotly.express as px

In [20]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [21]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
original_train_data = dpm.train_task1_df
original_train_data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [22]:
original_train_data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [23]:
colors = {0: 'royalblue', 1: 'coral'}

In [24]:
fig = px.bar(original_train_data['label'].value_counts(), x=['NO PCL', 'PCL'], y=original_train_data['label'].value_counts().values,
             labels={'x': 'Labels', 'y': 'Count'}, title='Distribution of Labels for the Entire Testing Set',
             color=['NO PCL', 'PCL'], color_discrete_sequence=list(colors.values()))
fig.update_layout(width=800,height=600)
fig.show()

In [32]:
color_discrete_map = {"No PCL": 'royalblue', "PCL": 'coral'}

df_grouped = original_train_data.groupby(['country', 'label']).size().reset_index(name='count')
df_grouped['label'] = df_grouped['label'].replace({0: "No PCL", 1: "PCL"})

fig = px.bar(df_grouped, x='country', y='count', color='label',
             title='Distribution of Labels Over Countries',
             barmode='group',
             color_discrete_map=color_discrete_map)

fig.update_layout(width=800, height=600)
fig.update_layout(legend_title_text='Label')

fig.show()

In [33]:
color_discrete_map = {"No PCL": 'royalblue', "PCL": 'coral'}

df_grouped = original_train_data.groupby(['keyword', 'label']).size().reset_index(name='count')
df_grouped['label'] = df_grouped['label'].replace({0: "No PCL", 1: "PCL"})

fig = px.bar(df_grouped, x='keyword', y='count', color='label',
             title='Distribution of Labels Over Keywords',
             barmode='group',
             color_discrete_map=color_discrete_map)

fig.update_layout(width=800, height=600)
fig.update_layout(legend_title_text='Label')

fig.show()

In [38]:
original_train_data['label'] = original_train_data['label'].replace({0: "No PCL", 1: "PCL"})

colors = {"No PCL": 'royalblue', "PCL": 'coral'}

fig_text_length = px.histogram(
    original_train_data, 
    x=original_train_data['text'].apply(len),
    color='label', 
    barmode='overlay',
    title='Distribution of Text Length by Label', 
    color_discrete_map=colors,
    labels={'x': 'Text Length', 'color': 'Label'}  # Updated labels mapping for x and color legend
)

fig_text_length.update_layout(width=800, height=600)
fig_text_length.show()

In [67]:
filtered_data = original_train_data[original_train_data['orig_label'] == "4"][['text', 'country']]
for item in filtered_data.values:
    print(item)

['"This is the rhetoric of what anthropologist Laura Agust ? n calls the "" rescue industry "" - a term used to describe people and institutions who conceptualize all sex workers as victims in need of saving . Catherine MacKinnon has argued , for example , that "" in prostitution , women have sex with men they would never otherwise have sex with ."'
 'za']
["When some people feel causing problem for some others by breaking into their homes to steal is n't too good , they just result to begging . You now see people without deformities begging , when some people who are disabled work to feed their mouth . You then ask , what type of country is Nigeria ? Even a man who is not lettered would chorus the maxim that two wrongs do n't make a right . The country is n't working out ; and people want to survive anyhow too . They have to eat they will say ."
 'ng']
['We are alarmed to learn of your recently circulated proposals that would eviscerate the Lifeline program and leave many of the most 